# Mapping and Perception for an autonomous robot (0510-7951)

### 2024/B

### Written by Roy Orfaig

---

SLAM: Simulatinoud Localization And Mapping!

-------------------------------------

**Instructions**

1. Fill in the code the "TODO" section
2. Answer the question inside the sections
3. **Please copy all the results to the report:**
  - Outputs- Images, tables, scores,etc
  - Performace, analysis and your explanations.
  - Attach the completed notebook to the report package.

Good luck!

In [1]:
import sys

project_path = r"C:\Users\Nadav\Desktop\TAU\מיפוי וחישה למערכות אוטונמיות\Project_3\EKF-SLAM"
sys.path.append(project_path)

import os
import io
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Ellipse
from PIL import Image
from tqdm import tqdm
import random
import pykitti
from math import sin, cos, sqrt

import graphs_slam
from graphs_slam import percentage_within_boundary,normalize_angle_radians,plot_state,save_graphs,plot_error

from data_preparation import *
from utils.ellipse import draw_prob_ellipse
from utils.draw_robot import draw_robot
from utils.misc_tools import error_ellipse
from utils.ellipse import draw_ellipse
from utils.misc_tools import error_ellipse
from data_loader_slam import DataSLAMLoader
from data_preparation import normalize_angle, normalize_angles_array
from utils import read_data

from utils.ellipse import draw_prob_ellipse
from utils.draw_robot import draw_robot
import pandas as pd

C:\Users\Nadav\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Student name+ID

### Nadav Marcinao

### 305165698

# Settings

In [2]:
#dont touch this function

In [2]:
class DataSLAMLoader:
    def __init__(self, basedir,dat_dir):
        self.dat_dir = dat_dir
    
    def load_landmarks(self):
        # read landmarks(SLAM)
        world_path = os.path.join(self.dat_dir, "world.dat")
        return read_data.read_world(world_path)
    
    def load_sensor_data(self):
        # read sensor data(SLAM)
        sensor_data_path = os.path.join(self.dat_dir, "sensor_data.dat")
        return read_data.read_sensor_data(sensor_data_path)
    

In [3]:
basedir = "/Users/Nadav/Desktop/TAU/מיפוי וחישה למערכות אוטונמיות/Project_3/EKF-SLAM" 
dat_dir = os.path.join(basedir,"data") 

dataset = DataSLAMLoader(basedir, dat_dir)

## Simultaneous localization and mapping  (SLAM)!

In [11]:
#Complete the #TODO sections within the ExtendedKalmanFilterSLAM class.

In [4]:
class ExtendedKalmanFilterSLAM:
    start_idx = 20 # ignore the first samples for the errors calculations
    def __init__(self, sigma_x_y_theta, variance_r1_t_r2, variance_r_phi, k):
        """
        Args:
            variance_x_y_theta: variance in x, y and theta respectively
            variance_r1_t_r2: variance in rotation1, translation and rotation2 respectively
            variance_r_phi: variance in the range and bearing
        """

        self.sigma_x_y_theta = sigma_x_y_theta
        self.variance_r_phi = variance_r_phi

        # additional params
        self.init_inf_val = 100 #TODO (hint - put value as the max size of the grid)
        self.R_tilde = np.diag(variance_r1_t_r2)

        self.k = k


    @staticmethod
    def calc_RMSE_maxE(X_Y_GT, X_Y_est):
        X_Y_diff = X_Y_GT[ExtendedKalmanFilterSLAM.start_idx:, :2] - X_Y_est[ExtendedKalmanFilterSLAM.start_idx:, :2]
        RMSE = np.linalg.norm(X_Y_diff) / np.sqrt(len(X_Y_GT) - ExtendedKalmanFilterSLAM.start_idx)
        maxE = np.max(np.abs(X_Y_diff[:,0]) + np.abs(X_Y_diff[:,1]))
        return RMSE, maxE ,X_Y_diff

    def predict(self, mu_prev, sigma_prev, u, N):
        # Perform the prediction step of the EKF
        # u[0]=translation, u[1]=rotation1, u[2]=rotation2

        delta_trans, delta_rot1, delta_rot2 = u['t'], u['r1'], u['r2']
        theta_prev = mu_prev[2]


        F = np.hstack((np.eye(3), np.zeros((3, 2*N))))
        G_x = np.eye(3) + np.array([[0, 0, -1*delta_trans*sin(theta_prev + delta_rot1)], [0, 0, delta_trans*cos(theta_prev + delta_rot1)], [0, 0, 0]])
        G = np.eye(3+2*N); G[0:3, 0:3] = G_x
        V = np.array([[-1*delta_trans*sin(theta_prev+delta_rot1), cos(theta_prev+delta_rot1), 0],
                      [delta_trans*cos(theta_prev+delta_rot1), sin(theta_prev+delta_rot1), 0],
                      [1, 0, 1]])


        # Calculate R with the added Q_additional
        R = F.T @ V @ self.R_tilde @ V.T @ F #TODO (hint: use F.T , V , self.R_tilde , V.T , F )

        odometry_model_next_step=np.array([[delta_trans*cos(theta_prev + delta_rot1)], [delta_trans*sin(theta_prev + delta_rot1)],[delta_rot1 + delta_rot2]])

        mu_est = mu_prev + F.T @ odometry_model_next_step
        sigma_est = G @ sigma_prev @ G.T + R #TODO (hint: use G , sigma_prev ,G.T,R)
        return mu_est, sigma_est


    def update(self, mu_pred, sigma_pred, z, observed_landmarks, N):
        # Perform filter update (correction) for each odometry-observation pair read from the data file.
        mu = mu_pred.copy()
        sigma = sigma_pred.copy()
        theta = mu[2][0] # the [0] is used to make theta a float and not an 1x1 array

        m = len(z["id"])
        Z = np.zeros(2 * m)
        z_hat = np.zeros(2 * m)
        H = None

        for idx in range(m):
            j = z["id"][idx] - 1
            r = z["range"][idx]
            phi = z["bearing"][idx]

            mu_j_x_idx = 3 + j*2
            mu_j_y_idx = 4 + j*2
            Z_j_x_idx = idx*2
            Z_j_y_idx = 1 + idx*2

            if observed_landmarks[j] == False:
                mu[mu_j_x_idx: mu_j_y_idx + 1] = mu[0:2] + np.array([r * cos(phi + theta), r * sin(phi + theta)]).reshape(-1, 1)
                observed_landmarks[j] = True

            Z[Z_j_x_idx : Z_j_y_idx + 1] = np.array([r, phi])

            delta = mu[mu_j_x_idx : mu_j_y_idx + 1] - mu[0 : 2]
            delta_x , delta_y = delta.ravel()
            q = float((delta.T) @ delta)
            # based on formula from lecture, delta_x, delta_y = delta; mu_bar_t_theta = mu[2]
            z_hat[Z_j_x_idx : Z_j_y_idx + 1] = np.array([sqrt(q), normalize_angle(np.arctan2(delta_y, delta_x)) - theta])

            I = np.diag(5*[1])
            F_j = np.hstack((I[:,:3], np.zeros((5, 2*j)), I[:,3:], np.zeros((5, 2*N-2*(j+1)))))

            sqrt_q = sqrt(q)
            Hi = (1/q) * np.array([[-1*sqrt_q*delta_x, -1*sqrt_q*delta_y, 0, sqrt_q*delta_x, sqrt_q*delta_y],[delta_y, -1*delta_x, -1*q, -1*delta_y, delta_x]]) @ F_j

            if H is None:
                H = Hi.copy()
            else:
                H = np.vstack((H, Hi))

        Q = self.k * np.diag(m * self.variance_r_phi)

        K = sigma_pred @ H.T @ np.linalg.inv(H @ sigma_pred @ H.T + Q) #TODO (hint- sigma_pred, H.T ,H , sigma_pred , H.T, Q)


        diff = Z - z_hat #TODO (hint- use Z , z_hat)
        diff[1::2] = normalize_angles_array(diff[1::2])

        mu =  mu + K @ diff.reshape(-1, 1) #TODO (hint- use mu,K, diff)
        sigma = (np.eye(sigma_pred.shape[0]) - K @ H) @ sigma_pred #TODO (hint- use K, H,sigma_pred)

        mu[2] = normalize_angle(float(mu[2]))

        return mu, sigma, observed_landmarks

    def run(self, sensor_data_gt, sensor_data_noised, landmarks, ax):
        # Get the number of landmarks in the map
        N = len(landmarks)

        # Initialize belief:
        # mu: 2N+3x1 vector representing the mean of the normal distribution
        # The first 3 components of mu correspond to the pose of the robot,
        # and the landmark poses (xi, yi) are stacked in ascending id order.
        # sigma: (2N+3)x(2N+3) covariance matrix of the normal distribution
        init_inf_val = self.init_inf_val
        mu_arr = np.zeros((1, 2*N+3))
        sigma_vec = np.concatenate((np.array(self.sigma_x_y_theta) ** 2, 2*N*[init_inf_val]))
        sigma_prev = np.diag(sigma_vec)

        # sigma for analysis graph sigma_x_y_t + select 2 landmarks
        landmark1_ind = 3 #TODO
        landmark2_ind = 13 #TODO

        Index=[0,1,2,landmark1_ind,landmark1_ind+1,landmark2_ind,landmark2_ind+1]
        sigma_x_y_t_px1_py1_px2_py2 = sigma_prev[Index,Index].copy()

        observed_landmarks = np.zeros(N, dtype=bool)

        sensor_data_count = int(len(sensor_data_noised) / 2)
        frames = []

        mu_arr_gt = np.array([[0, 0, 0]])

        for idx in range(sensor_data_count):
            mu_prev = mu_arr[-1].reshape(-1, 1)

            u = sensor_data_noised[(idx, "odometry")]
            # predict
            mu_pred, sigma_pred = self.predict(mu_prev, sigma_prev, u, N)
            # update (correct)
            mu, sigma, observed_landmarks = self.update(mu_pred, sigma_pred, sensor_data_noised[(idx, "sensor")], observed_landmarks, N)

            mu_arr = np.vstack((mu_arr, mu.T))
            sigma_prev = sigma.copy()
            sigma_x_y_t_px1_py1_px2_py2 = np.vstack((sigma_x_y_t_px1_py1_px2_py2, sigma_prev[Index,Index].copy()))

            delta_r1_gt = sensor_data_gt[(idx, "odometry")]["r1"]
            delta_r2_gt = sensor_data_gt[(idx, "odometry")]["r2"]
            delta_trans_gt = sensor_data_gt[(idx, "odometry")]["t"]

            calc_x = lambda theta_p: delta_trans_gt * cos(theta_p + delta_r1_gt)
            calc_y = lambda theta_p: delta_trans_gt * sin(theta_p + delta_r1_gt)

            theta = delta_r1_gt + delta_r2_gt

            theta_prev = mu_arr_gt[-1,2]
            mu_arr_gt = np.vstack((mu_arr_gt, mu_arr_gt[-1] + np.array([calc_x(theta_prev), calc_y(theta_prev), theta])))

            if ax is not None:
                frame = plot_state(ax, mu_arr_gt, mu_arr, sigma, landmarks, observed_landmarks, sensor_data_noised[(idx, "sensor")])
                frames.append(frame)

        return frames, mu_arr, mu_arr_gt, sigma_x_y_t_px1_py1_px2_py2


In [5]:
#Complete the #TODO sections within the ProjectQuestion class.
class ProjectQuestion:
    def __init__(self, dataset,save_results,save_animation):

        self.dataset = dataset
        self.save_results=save_results
        self.save_animation=save_animation
        self.fig_dir_path = os.getcwd() + "/Results_EKFSLAM/"
        if not os.path.exists(self.fig_dir_path):
            os.makedirs(self.fig_dir_path)


    def get_odometry(self, sensor_data):
        """
        Args:
            sensor_data: map from a tuple (frame number, type) where type is either ‘odometry’ or ‘sensor’.
            Odometry data is given as a map containing values for ‘r1’, ‘t’ and ‘r2’ – the first angle, the translation and the second angle in the odometry model respectively.
            Sensor data is given as a map containing:
              - ‘id’ – a list of landmark ids (starting at 1, like in the landmarks structure)
              - ‘range’ – list of ranges, in order corresponding to the ids
              - ‘bearing’ - list of bearing angles in radians, in order corresponding to the ids

        Returns:
            numpy array of dim [num of frames X 3]
            first two components in each row are the x and y in meters
            the third component is the heading in radians
        """
        num_frames = len(sensor_data) // 2
        state = np.array([[0, 0, 0]], dtype=float).reshape(1, 3)
        for i in range(num_frames):
            curr_odometry = sensor_data[i, 'odometry']
            t = np.array([
                curr_odometry['t'] * np.cos(state[-1, 2] + curr_odometry['r1']),
                curr_odometry['t'] * np.sin(state[-1, 2] + curr_odometry['r1']),
                curr_odometry['r1'] + curr_odometry['r2']
            ]).reshape(3, 1)
            new_pos = state[-1, :].reshape(3, 1) + t
            state = np.concatenate([state, new_pos.reshape(1, 3)], axis=0)
        return state

    def calc_trajectory_from_odometry(self, sensor_data):
        num_of_frames = int(len(sensor_data) / 2) # half of the list is odometry and half are sensors

        # initialize the pose of the robot
        poses = [np.array([0, 0, 0])]

        # iterate over frames
        for idx in range(num_of_frames):
            delta_r1 = sensor_data[(idx, "odometry")]["r1"]
            delta_r2 = sensor_data[(idx, "odometry")]["r2"]
            delta_t = sensor_data[(idx, "odometry")]["t"]

            prev_pose = poses[-1]
            curr_pose = [prev_pose[0] + delta_t * cos(prev_pose[2] + delta_r1),
                         prev_pose[1] + delta_t * sin(prev_pose[2] + delta_r1),
                         prev_pose[2] + delta_r1 + delta_r2]

            poses.append(curr_pose)

        return np.array(poses)

    def Q1(self):

        """
        Runs the code for question 3 of the project
        Loads the odometry (robot motion) and sensor (landmarks) data supplied with the exercise
        Adds noise to the odometry data r1, trans and r2
        Uses the extended Kalman filter SLAM algorithm with the noisy odometry data to predict the path of the robot and
        the landmarks positions
        """

        ### a. load the data ###
        print("Start EKF-SLAM.., please wait...")
        # Pre-processing
        landmarks = self.dataset.load_landmarks()
        sensor_data_gt = self.dataset.load_sensor_data()
        N = len(landmarks)

        variance_r1_t_r2 = [0.01 ** 2, 0.1 ** 2, 0.01 ** 2] #TODO  (hint- this is varinace not std..)
        sigma_x_y_theta = [np.sqrt(variance_r1_t_r2[1]), np.sqrt(variance_r1_t_r2[1]), np.sqrt(variance_r1_t_r2[0]+variance_r1_t_r2[2])]
        Q_additional = np.diag([0, 0, 0]) # Adding an additional system noise matrix
        
        #hint 1: use variance_r1_t_r2[1] and variance_r1_t_r2[0]+variance_r1_t_r2[2] ,
        #hint 2: use std not variance) ) [np.sqrt(variance_r1_t_r2[1]), np.sqrt(variance_r1_t_r2[1]), np.sqrt(variance_r1_t_r2[0]+variance_r1_t_r2[2])]

        variance_r_phi = [0.3 ** 2, 0.2 ** 2] #TODO (hint- this is varinace not std..)

        ### b - c. add noise to the sensor data and plot it vs the GT ###

        # add some noise to the data
        sensor_data_noised = add_gaussian_noise_dict(sensor_data_gt, list(np.sqrt(np.array(variance_r1_t_r2))))

        # calculate trajectories
        pos_xy_gt = self.calc_trajectory_from_odometry(sensor_data_gt)
        pos_xy_noise = self.calc_trajectory_from_odometry(sensor_data_noised)

        # plot trajectory
        graphs_slam.plot_trajectory(pos_xy_gt, 'GT trajectory from odometry', 'x [m]', 'y [m]')
        save_graphs(self.fig_dir_path,"GT trajectory from odometry")

        # plot trajectory + noise
        graphs_slam.plot_trajectory_with_noise(pos_xy_gt, pos_xy_noise, 'GT vs noisy trajectory from odometry', 'x [m]', 'y [m]')
        save_graphs(self.fig_dir_path,"GT vs noisy trajectory from odometry")
        fig_anim = plt.figure()
        ax = fig_anim.add_subplot(111)
          # Add grid
        ax.grid(True)

        # Set title
        ax.set_title("Extended Kalman Filter SLAM")

        # Set axis labels
        ax.set_xlabel("X")
        ax.set_ylabel("Y")
        
        ### Improving parameters and finding K

        k_vector = np.arange(1, 41, 1)
        Rmse_vec = []
        maxE_vec = []
        percentages_x = []
        percentages_y = []
        percentages_yaw = []

        # Define your boundaries

        for k in k_vector:
            ekf_slam = ExtendedKalmanFilterSLAM(sigma_x_y_theta, variance_r1_t_r2, variance_r_phi, k=k)

            # Run SLAM
            _, mu_arr, mu_arr_gt, sigma_x_y_t_px1_py1_px2_py2 = ekf_slam.run(sensor_data_gt, sensor_data_noised, landmarks, ax=None)  # ax=None -> no plots

            cov_x = np.sqrt(sigma_x_y_t_px1_py1_px2_py2[:,0])
            cov_y = np.sqrt(sigma_x_y_t_px1_py1_px2_py2[:,1])
            cov_yaw = np.sqrt(sigma_x_y_t_px1_py1_px2_py2[:,2])
            
            # Compute RMSE and max error
            RMSE, maxE, X_Y_diff = ExtendedKalmanFilterSLAM.calc_RMSE_maxE(mu_arr, mu_arr_gt)

            # Compute error vectors
            x_errors = mu_arr_gt[:, 0] - mu_arr[:, 0]
            y_errors = mu_arr_gt[:, 1] - mu_arr[:, 1]
            yaw_errors = np.vectorize(normalize_angle)(mu_arr_gt[:, 2] - mu_arr[:, 2])

            # Compute error percentages
            percentage_x = percentage_within_boundary(x_errors, cov_x)
            percentage_y = percentage_within_boundary(y_errors, cov_y)
            percentage_yaw = percentage_within_boundary(yaw_errors, cov_yaw)

            # Store results
            Rmse_vec.append(RMSE)
            maxE_vec.append(maxE)
            percentages_x.append(percentage_x)
            percentages_y.append(percentage_y)
            percentages_yaw.append(percentage_yaw)

        # Create a DataFrame
        results_df = pd.DataFrame({
        'k': k_vector,
        'RMSE [m]': Rmse_vec,
        'maxE [m]': maxE_vec,
        'Percentage X within boundary [%]': percentages_x,
        'Percentage Y within boundary [%]': percentages_y,
        'Percentage YAW within boundary [%]': percentages_yaw})

        # Optionally, save the results to a CSV file
        if self.save_results:
            results_df.to_csv(f"{self.fig_dir_path}/error_analysis.csv", index=False)
            print("Saving error analysis")

        ### d - h. ExtendedKalmanFilterSLAM ###
        
        k_vector = np.arange(1, 10.10, 0.10)
        Rmse_vec = []; maxE_vec = []
        for k in k_vector:
            ekf_slam = ExtendedKalmanFilterSLAM(sigma_x_y_theta, variance_r1_t_r2, variance_r_phi, k=k)

            # frames, mu_arr, mu_arr_gt, sigma_x_y_t_px1_py1_px2_py2 = ekf_slam.run(sensor_data_gt, sensor_data_noised, landmarks, ax=None) # ax = None -> no plots
            _, mu_arr, mu_arr_gt, _ = ekf_slam.run(sensor_data_gt, sensor_data_noised, landmarks, ax=None) # ax = None -> no plots

            RMSE, maxE ,X_Y_diff = ExtendedKalmanFilterSLAM.calc_RMSE_maxE(mu_arr, mu_arr_gt)

            Rmse_vec.append(RMSE)
            maxE_vec.append(maxE)

        min_idx_rmse = np.argmin(Rmse_vec)
        min_idx_maxE = np.argmin(maxE_vec)

        fig_calib, axes_calib = plt.subplots(2,1)
        axes_calib[0].plot(k_vector, Rmse_vec)
        axes_calib[0].scatter(k_vector[min_idx_rmse], Rmse_vec[min_idx_rmse])
        axes_calib[0].set_title('RMSE vs sigma_theta values \n min for k = '+str(round(k_vector[min_idx_rmse], 3)))
        axes_calib[0].set(xlabel='sigma_theta', ylabel='Rmse')
        axes_calib[0].grid()
        axes_calib[1].plot(k_vector, maxE_vec)
        axes_calib[1].scatter(k_vector[min_idx_maxE], maxE_vec[min_idx_maxE])
        axes_calib[1].set_title('maxE vs sigma_n values \n min for k = '+str(round(k_vector[min_idx_maxE],3)))
        axes_calib[1].set(xlabel='sigma_theta', ylabel='maxE')
        axes_calib[1].grid()
        fig_calib.tight_layout()

        save_graphs(self.fig_dir_path,"Extended Kalman filter SLAM calibration")

        opt_k = k_vector[min_idx_rmse]
        
        ekf_slam = ExtendedKalmanFilterSLAM(sigma_x_y_theta, variance_r1_t_r2, variance_r_phi, k=opt_k)
        frames, mu_arr, mu_arr_gt, sigma_x_y_t_px1_py1_px2_py2 = ekf_slam.run(sensor_data_gt, sensor_data_noised, landmarks, ax)
        RMSE, maxE,X_Y_diff = ExtendedKalmanFilterSLAM.calc_RMSE_maxE(mu_arr, mu_arr_gt)
        print("EKF-SLAM: RMSE = " + str(round(RMSE, 3))+" [m], maxE = "+str(round(maxE,3))+" [m]")

        # save the analysis figures
        if self.save_results:
            fig, ax = plt.subplots(1, 1, figsize=(8, 6))
            fig.suptitle("Extended Kalman Filter SLAM Trajectory Results \n k = " + str(opt_k) + "\nRMSE = " + str(round(RMSE, 3)) + ", maxE = "+str(round(maxE,3)))
            ax.plot(mu_arr_gt[:,0], mu_arr_gt[:,1], label='GT')
            ax.plot(mu_arr[:,0], mu_arr[:,1], label='EKF-SLAM')
            ax.set(xlabel='x[m]', ylabel='y[m]')
            ax.grid()
            ax.legend()

            save_graphs(self.fig_dir_path,"Extended Kalman SLAM filter results vs GT")
            angle_error = np.vectorize(normalize_angle)(mu_arr_gt[:,2] - mu_arr[:,2])
            plot_error([mu_arr_gt[:,0] - mu_arr[:,0],np.sqrt(sigma_x_y_t_px1_py1_px2_py2[:,0])],[mu_arr_gt[:,1] - mu_arr[:,1],np.sqrt(sigma_x_y_t_px1_py1_px2_py2[:,1])],[angle_error,np.sqrt(sigma_x_y_t_px1_py1_px2_py2[:,2])])
            save_graphs(self.fig_dir_path,"state_vector_Error- GT - EKFSLAM")

            Lk1x=[np.tile(landmarks[1][0], mu_arr.shape[0]) - mu_arr[:,3],np.sqrt(sigma_x_y_t_px1_py1_px2_py2[:,3])]
            Lk1y=[np.tile(landmarks[1][1], mu_arr.shape[0]) - mu_arr[:,4],np.sqrt(sigma_x_y_t_px1_py1_px2_py2[:,4])]
            plot_error(Lk1x,Lk1y)
            save_graphs(self.fig_dir_path,"landmark1_Error- GT - EKFSLAM")

            Lk2x=[np.tile(landmarks[2][0], mu_arr.shape[0]) - mu_arr[:,5],np.sqrt(sigma_x_y_t_px1_py1_px2_py2[:,5])]
            Lk2y=[np.tile(landmarks[2][1], mu_arr.shape[0]) - mu_arr[:,6],np.sqrt(sigma_x_y_t_px1_py1_px2_py2[:,6])]
            plot_error(Lk2x,Lk2y)
            save_graphs(self.fig_dir_path,"landmark2_Error- GT - EKFSLAM")



        ax.set_xlim([-2, 12])
        ax.set_ylim([-2, 12])
         # save the animation
        if self.save_animation:
            print("Saving animation..wait...")
            ani = animation.ArtistAnimation(fig_anim, frames, repeat=False)
            ani.save(self.fig_dir_path +'Extended Kalman Filter SLAM.mp4', metadata={'artist':'me'})
            graphs_slam.show_graphs()
            print("Done!...")

    def run(self):
        self.Q1()
        print("Successfully finished. All data saved in {}".format(self.fig_dir_path))


# Run the program

In [6]:
seed = 50 #dont change seed number
np.random.seed(seed)

save_results=True
save_animation=True

project = ProjectQuestion(dataset,save_results,save_animation)
project.run()

Start EKF-SLAM.., please wait...


C:\Users\Nadav\AppData\Local\Temp\ipykernel_22208\2684809251.py:37: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  G_x = np.eye(3) + np.array([[0, 0, -1*delta_trans*sin(theta_prev + delta_rot1)], [0, 0, delta_trans*cos(theta_prev + delta_rot1)], [0, 0, 0]])
C:\Users\Nadav\AppData\Local\Temp\ipykernel_22208\2684809251.py:39: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  V = np.array([[-1*delta_trans*sin(theta_prev+delta_rot1), cos(theta_prev+delta_rot1), 0],
C:\Users\Nadav\AppData\Local\Temp\ipykernel_22208\2684809251.py:40: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure 

Saving error analysis
EKF-SLAM: RMSE = 0.2 [m], maxE = 0.594 [m]
Saving animation..wait...
Done!...
Successfully finished. All data saved in C:\Users\Nadav/Results_EKFSLAM/
